# Modelo de Reconocimiento de Platos Bolivianos por Imagen

Este notebook desarrolla un modelo de visión por computadora para reconocer platos bolivianos a partir de imágenes y obtener su información nutricional.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import glob

# Importar utilidades
import sys
import os
sys.path.append(os.path.join(os.path.dirname("__file__"), '..', 'utils'))

from cargar_datos import cargar_datos_ingredientes, cargar_datos_nutricion, procesar_ingredientes, combinar_datos
from preprocesamiento import crear_etiquetas_nutricionales

In [ ]:
# Cargar los datos de ingredientes y nutrición
df_ingredientes = cargar_datos_ingredientes('../dataset_ingredientes.csv')
df_nutricion = cargar_datos_nutricion('../nutricion/datos_nutricionales.csv')
print(f"Dataset de ingredientes: {df_ingredientes.shape}")
print(f"Dataset nutricional: {df_nutricion.shape}")

# Combinar datasets
df_completo = combinar_datos(df_ingredientes, df_nutricion)
df_completo = crear_etiquetas_nutricionales(df_completo)
print(f"Dataset combinado: {df_completo.shape}")
print(df_completo[['plato', 'calorias', 'proteina', 'carbohidratos', 'grasa']].head())

In [ ]:
# Crear listas para almacenar rutas de imágenes y etiquetas de platos
imagenes_paths = []
etiquetas_platos = []

# Directorio de imágenes
dataset_dir = '../dataset_imagenes/'

# Recorrer cada subdirectorio (plato)
for plato_dir in os.listdir(dataset_dir):
    plato_path = os.path.join(dataset_dir, plato_dir)
    
    if os.path.isdir(plato_path):
        # Convertir nombre de directorio al formato del dataset
        nombre_plato = plato_dir.replace('_', ' ').title()
        # Correcciones especiales para ciertos platos
        if 'Orureno' in nombre_plato:
            nombre_plato = nombre_plato.replace('Orureno', 'Orureño')
        if 'Beniano' in nombre_plato:
            nombre_plato = nombre_plato.replace('Beniano', 'Beniano')
        if 'Pique Macho' in nombre_plato:
            nombre_plato = 'Pique Macho'
        elif 'Aji De Pataskha' in nombre_plato:
            nombre_plato = 'Aji de Pataskha'
        elif 'Caldo De Bagre' in nombre_plato:
            nombre_plato = 'Caldo de Bagre'
        
        # Verificar si el plato existe en nuestro dataset de ingredientes
        if nombre_plato in df_completo['plato'].values:
            # Obtener todas las imágenes del plato
            for img_file in os.listdir(plato_path):
                if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    imagenes_paths.append(os.path.join(plato_path, img_file))
                    etiquetas_platos.append(nombre_plato)

print(f"Número total de imágenes encontradas: {len(imagenes_paths)}")
print(f"Número total de etiquetas: {len(etiquetas_platos)}")
print(f"Número de platos únicos: {len(set(etiquetas_platos))}")
print(f"Ejemplos de platos: {list(set(etiquetas_platos))[:5]}")

In [ ]:
# Codificar las etiquetas de platos
label_encoder = LabelEncoder()
etiquetas_codificadas = label_encoder.fit_transform(etiquetas_platos)
num_clases = len(label_encoder.classes_)

print(f"Número de clases (platos diferentes): {num_clases}")
print(f"Clases (nombres de platos): {label_encoder.classes_[:5]}...")

# Convertir a categorías para el modelo
y = to_categorical(etiquetas_codificadas, num_classes=num_clases)
print(f"Forma de las etiquetas codificadas: {y.shape}")

In [ ]:
# Dividir los datos
X_paths = np.array(imagenes_paths)
y_array = np.array(y)

# Dividir en entrenamiento, validación y prueba
X_temp, X_test, y_temp, y_test = train_test_split(X_paths, y_array, test_size=0.2, random_state=42, stratify=etiquetas_codificadas)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)

print(f"Entrenamiento: {X_train.shape[0]} imágenes")
print(f"Validación: {X_val.shape[0]} imágenes")
print(f"Prueba: {X_test.shape[0]} imágenes")

In [ ]:
# Definir dimensiones y parámetros
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 16  # Reducir batch size para manejar la memoria

# Función para cargar y preprocesar imágenes
def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is not None:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH))
        image = image.astype(np.float32) / 255.0
    else:
        print(f"Advertencia: No se pudo cargar la imagen {image_path}")
        # Crear una imagen vacía como fallback
        image = np.zeros((IMG_HEIGHT, IMG_WIDTH, 3), dtype=np.float32)
    return image

# Generador de datos personalizado
class PlatoImageGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_paths, labels, batch_size, shuffle=True):
        self.image_paths = image_paths
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.image_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)
            
    def __len__(self):
        return int(np.ceil(len(self.image_paths) / self.batch_size))
    
    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        
        batch_paths = [self.image_paths[i] for i in indexes]
        batch_labels = np.array([self.labels[i] for i in indexes])
        
        batch_images = np.array([load_and_preprocess_image(path) for path in batch_paths])
        
        return batch_images, batch_labels
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

In [ ]:
# Crear generadores para entrenamiento, validación y prueba
train_generator = PlatoImageGenerator(X_train, y_train, BATCH_SIZE, shuffle=True)
val_generator = PlatoImageGenerator(X_val, y_val, BATCH_SIZE, shuffle=False)
test_generator = PlatoImageGenerator(X_test, y_test, BATCH_SIZE, shuffle=False)

print("Generadores creados exitosamente")

In [ ]:
# Crear modelo usando transfer learning con VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

# Congelar capas del modelo base
base_model.trainable = False

# Añadir capas personalizadas para clasificación de platos bolivianos
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_clases, activation='softmax')  # Clasificación multiclase para identificar platos
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(model.summary())

In [ ]:
# Definir callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=1e-7
)

# Entrenar el modelo
EPOCHS = 30  # Número de épocas razonable

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

In [ ]:
# Visualizar métricas del entrenamiento
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.plot(history.history['loss'], label='Pérdida Entrenamiento')
ax1.plot(history.history['val_loss'], label='Pérdida Validación')
ax1.set_title('Pérdida durante el entrenamiento')
ax1.set_xlabel('Época')
ax1.set_ylabel('Pérdida')
ax1.legend()

ax2.plot(history.history['accuracy'], label='Precisión Entrenamiento')
ax2.plot(history.history['val_accuracy'], label='Precisión Validación')
ax2.set_title('Precisión durante el entrenamiento')
ax2.set_xlabel('Época')
ax2.set_ylabel('Precisión')
ax2.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Evaluar el modelo en datos de prueba
test_loss, test_accuracy = model.evaluate(test_generator, verbose=0)
print(f"Precisión en datos de prueba: {test_accuracy:.4f}")
print(f"Pérdida en datos de prueba: {test_loss:.4f}")

In [ ]:
# Función para predecir el plato a partir de una imagen
def predecir_plato(imagen_path, model, label_encoder):
    # Cargar y preprocesar la imagen
    img = load_and_preprocess_image(imagen_path)
    img = np.expand_dims(img, axis=0)  # Agregar dimensión de batch
    
    # Hacer predicción
    predicciones = model.predict(img)
    prediccion_idx = np.argmax(predicciones[0])
    prediccion_plato = label_encoder.classes_[prediccion_idx]
    confianza = predicciones[0][prediccion_idx]
    
    return prediccion_plato, confianza

# Función para obtener información nutricional de un plato
def obtener_info_nutricional(plato, df_completo):
    info = df_completo[df_completo['plato'] == plato]
    if not info.empty:
        return {
            'calorias': info['calorias'].iloc[0],
            'proteina': info['proteina'].iloc[0],
            'carbohidratos': info['carbohidratos'].iloc[0],
            'grasa': info['grasa'].iloc[0],
            'categoria_calorias': info['categoria_calorias'].iloc[0],
            'categoria_proteina': info['categoria_proteina'].iloc[0],
            'categoria_carbohidratos': info['categoria_carbohidratos'].iloc[0],
            'categoria_grasa': info['categoria_grasa'].iloc[0],
            'ingredientes': info['lista_ingredientes'].iloc[0]
        }
    else:
        return None

# Función para análisis completo de imagen
def analizar_imagen_completo(imagen_path, model, label_encoder, df_completo):
    # Predecir el plato
    plato_predicho, confianza = predecir_plato(imagen_path, model, label_encoder)
    
    # Obtener información nutricional
    info_nutricional = obtener_info_nutricional(plato_predicho, df_completo)
    
    print(f"Imagen analizada: {os.path.basename(imagen_path)}")
    print(f"Plato identificado: {plato_predicho}")
    print(f"Confianza de la predicción: {confianza:.4f}")
    
    if info_nutricional:
        print("\nInformación nutricional:")
        print(f"  - Calorías: {info_nutricional['calorias']} kcal ({info_nutricional['categoria_calorias']})")
        print(f"  - Proteína: {info_nutricional['proteina']}g ({info_nutricional['categoria_proteina']})")
        print(f"  - Carbohidratos: {info_nutricional['carbohidratos']}g ({info_nutricional['categoria_carbohidratos']})")
        print(f"  - Grasa: {info_nutricional['grasa']}g ({info_nutricional['categoria_grasa']})")
        print(f"  - Ingredientes: {', '.join(info_nutricional['ingredientes'][:5])}...")  # Mostrar primeros 5
    else:
        print("\nNo se encontró información nutricional para este plato.")
    
    return plato_predicho, info_nutricional

In [ ]:
# Probar con una imagen de ejemplo
if len(X_test) > 0:
    ejemplo_imagen = X_test[0]
    print("Probando con una imagen de ejemplo del conjunto de prueba:\n")
    
    plato_predicho, info_nutricional = analizar_imagen_completo(
        ejemplo_imagen, model, label_encoder, df_completo
    )

In [ ]:
# Mostrar algunas imágenes con sus predicciones
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for i in range(min(6, len(X_test))):
    img_path = X_test[i]
    img = cv2.imread(img_path)
    if img is not None:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        axes[i].imshow(img_rgb)
        
        # Hacer predicción
        plato_predicho, confianza = predecir_plato(img_path, model, label_encoder)
        
        axes[i].set_title(f"Predicción: {plato_predicho[:10]}...\nConfianza: {confianza:.2f}")
        axes[i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Guardar el modelo entrenado
model.save('modelo_reconocimiento_platos.h5')
print("Modelo guardado como 'modelo_reconocimiento_platos.h5'")

# Guardar el label encoder
import pickle
with open('label_encoder_platos.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)
print("Label encoder guardado como 'label_encoder_platos.pkl'")

In [ ]:
# Función para usar el modelo en una imagen nueva
def usar_modelo_en_imagen_nueva(imagen_path, modelo_path='modelo_reconocimiento_platos.h5', 
                                 encoder_path='label_encoder_platos.pkl', df_completo=df_completo):
    from tensorflow.keras.models import load_model
    import pickle
    
    # Cargar el modelo entrenado
    model = load_model(modelo_path)
    
    # Cargar el label encoder
    with open(encoder_path, 'rb') as f:
        label_encoder = pickle.load(f)
    
    # Analizar la imagen
    plato_predicho, info_nutricional = analizar_imagen_completo(
        imagen_path, model, label_encoder, df_completo
    )
    
    return plato_predicho, info_nutricional

# Ejemplo de cómo usar con una imagen nueva:
# plato, info = usar_modelo_en_imagen_nueva('ruta/a/tu/imagen.jpg')